# Récupérer les métadonnées d'un article pour enregistrer son DOI chez Crossref

Contexte : un article de revue est publié et on souhaite enregistrer son DOI chez CrossRef. Le script python récupère les métadonnées de l'article publié, les ré-organise de sorte à former un fichier XML conforme au schéma CrossRef afin de pouvoir enregistrer le DOI de l'article. Les métadonnées récupérées son : titre, auteurs, année, DOI, résumé et références biblio. Le fichier XML est ensuite à charger dans [doi.crossref.org](https://doi.crossref.org)

Script utilisé avec des articles publiés par Lodel 1.0 <br /> 
2023-07, Maxence Larrieu

In [21]:
import requests
from  bs4 import BeautifulSoup

In [33]:
## renseigner l'URL de l'article dont il faut enregistrer le DOI
lien_article = "https://revues.mshparisnord.fr/rfim/index.php?id=479"

## 1.  Récupérer les métadonnées depuis la page d'un l'article

In [34]:
## un dictionnaire pour coller les métadonnées
article_md = {}

In [35]:
# Entête de la requête
entete = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
}

In [36]:
## recueillir toutes les données de la page
response = requests.get(lien_article, headers=entete)

## les placer dans Beautiful soup
soup = BeautifulSoup(response.content, "html.parser")

### Cibler les métadonnées à récupérer

In [37]:
# Titre de l'article
article_md["title"] = soup.find("title").text.strip()

In [38]:
## récupérer tous les auteurs
auth_list = soup.find_all("meta", {"name": "author"})
auths = {}

## passer la liste au niveau unitaire
idx = 0
for item in auth_list :
    full_name = item.get("content").strip()
    auths[idx] = {}
    auths[idx]["given_name"] = full_name[ full_name.index(",") +2 :]
    auths[idx]["surname"] = full_name[ : full_name.index(",")]
    idx += 1

article_md["auths"] = auths

In [39]:
## abstract
article_md["abstract"] = soup.find("meta", {"name" : "DC.Description"}).get("content")

In [40]:
## date
full_date = soup.find("div", {"id" : "date"}).text
article_md["date"] = full_date.split(" ")[1]
if not article_md["date"].isnumeric() : 
    print("//!\\ probleme when getting date")
    quit()

In [41]:
## doi
doi_raw = soup.find("p", {"class" : "article__doi"}).text
doi_raw.split(" ")[1]
article_md["doi"] = doi_raw[ doi_raw.index("doi.org/") + 8 :]


In [42]:
## print article metadatas
for item in article_md : 
    print(item, "\t", article_md[item], "\n")

title 	 Des outils interactifs pour l’étude des méthodes de composition des musiques électroacoustiques : les logiciels TaCEM comme ressources pédagogiques 

auths 	 {0: {'given_name': 'Frédéric', 'surname': 'Dufeu'}, 1: {'given_name': 'Peter', 'surname': 'Manning'}, 2: {'given_name': 'Michael', 'surname': 'Clarke'}} 

abstract 	 Introduction L’enseignement et l’apprentissage de la musique liée aux technologies, et plus particulièrement au traitement numérique de signal, peuvent dans une large mesure bénéficier de la recherche sur la production des musiques électroacoustiques. En effet, ce répertoire repose par définition sur un usage substantiel des nouvelles technologies, et l’étude d’une situation compositionnelle donnée offre un point de vue privilégié sur, d’une part, un processus créatif singulier et, d’autre part, un ensemble de techniques pour la génération et la transformation du son et des matériaux musicaux. Une approche par études de cas peut ainsi compléter l’apprentissage

In [43]:
## extra : get biblio
biblio_raw = soup.find_all("p", {"class" : "bibliographie"})
biblio = []
for item in biblio_raw : 
    biblio.append(item.get_text())

biblio[:2]

['Battier Marc, Cheret Bertrand, Lemouton Serge & Philippe Manoury (dir.) (2003), PMA LIB. Les musiques électroniques de Philippe Manoury, cédérom multiplateforme, Paris, Ircam-Centre Pompidou.',
 'Baudouin Olivier (2009), «\xa0A Reconstruction of Stria\xa0», Computer Music Journal, vol.\xa031, nº\xa03, p.\xa075-81.']

## Load local XML and parse article metadatas

In [44]:
## charger un fichier XML exemple que l'on modifiera par la suite
with open('crossref-example-to-edit.xml', 'r') as f:
    file = f.read()
    crossref = BeautifulSoup(file, 'html.parser')

c:\users\agentup\appdata\local\programs\python\python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [45]:
## xml buffer pour un auteur
xml_author = '''<person_name contributor_role="author" language="en" sequence="first">
<given_name>Malek</given_name>
<surname>Mansour</surname>
</person_name>'''

In [46]:
## ajout des auteurs dans le XML d'exemple

for auth_idx in article_md["auths"] :
    
    # load xml template for one authors
    auth_soup = BeautifulSoup(xml_author, "html.parser")
    ## print(auth_idx)
    
    # change name and surname
    auth_soup.given_name.string.replace_with( article_md["auths"][auth_idx]["given_name"])
    auth_soup.surname.string.replace_with( article_md["auths"][auth_idx]["surname"])
    
    # inclure additionnal as value attribute
    if auth_idx > 0 : 
        pers_name = auth_soup.find("person_name")
        pers_name["sequence"] = "additional"       
    
    # paste data into the XML tree
    crossref.contributors.append(auth_soup)
    

In [47]:
## update doi
crossref.doi.string.replace_with(article_md["doi"])
#crossref.doi

'10.53480/emerg-neurol.5410'

In [48]:
## update ressource link
crossref.resource.string.replace_with(lien_article)

'https://emerging-neurologist.org/article/view/32'

In [49]:
## update title
crossref.title.string.replace_with(article_md["title"])

'Unusual flare up of Susac syndrome'

In [50]:
## update year
crossref.year.string.replace_with(article_md["date"])
crossref.year

<year>2018</year>

In [51]:
## update abstract
crossref.find("jats:p").string.replace_with( article_md["abstract"])

'Introduction. Spinal cord involvement as a flare-up of Susac syndrome (SuS) is rarely reported in the literature. The pathogenesis of SuS includes multiple occlusions in micro vessels mediated by an autoimmune response to unknown antigens. This condition is characterized by a triad: central nervous system (CNS) dysfunction with a frequent involvement of the corpus callosum in brain MRI, visual disturbances due to branch retinal artery occlusion, and sensorineural hearing impairment.\r\nCase report. A 50-year-old man presented in December 2019 symptoms associating gait instability, diplopia and hypoacusis. A year later, he developed a vesical-sphincter disorder and a hypoesthesia in his right lower limb. Neurological examination identified a medullary syndrome, cerebellar ataxia and hypoacusis. We performed a brain and spinal MRI showing supratentorial and infratentorial white matter lesions, and gadolinium enhancing cervical lesions. Lumbar puncture with isoelectric focusing results w

In [52]:
## add bibliography

In [53]:
## xml buffer for one ref
xml_ref = '''<citation key="">
<unstructured_citation>one_ref_content</unstructured_citation>
</citation>'''

In [22]:
for idx, ref in enumerate(biblio) : 
    ## laod xml template for one ref
    ref_soup = BeautifulSoup(xml_ref, "html.parser")
    
    ## change ref content
    ref_soup.unstructured_citation.string.replace_with(ref)
    
    ## update idex attribute value
    ref_temp = ref_soup.find("citation")
    ref_temp["key"] = idx
    
    # paste data into the XML tree
    crossref.citation_list.append(ref_soup)

In [55]:
## export XML document

xml_content = str(crossref)

## enregistre le fichier XML sous la forme crossref-auto-273.xml où 273 est l'identifiant de l'URL
with open(f"crossref-auto-{lien_article[-3:]}.xml", "w", encoding="utf-8") as xml_file : 
    xml_file.write(xml_content)



In [56]:
xml_file.close()